In [ ]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.evapo import *

workhorse = True

if workhorse:
    origin = 'Aldhani/eoagritwin/'
else:
    origin = ''

import re
from other_repos.pyDMS.pyDMS.pyDMS import *

int_to_Month = {
    '01': 'January',
    '02': 'February',
    '03': 'March',
    '04': 'April',
    '05': 'May',
    '06': 'June',
    '07': 'July',
    '08': 'August',
    '09': 'September',
    '10': 'October',
    '11': 'November',
    '12': 'December'
    }

dayCount_leap = [31,29,31,30,31,30,31,31,30,31,30,31]
dayCount_noleap = [31,28,31,30,31,30,31,31,30,31,30,31]

In [ ]:
base_path = f'/data/{origin}force/output/Guzinski/'
# define the year you are working on 
year = 2019

# first, all files for months outside of studay aim will be deleted to save storgage
reduce_forceTSI_output_to_validmonths(f'{base_path}{year}/tiles', 4, 10)
print('All FORCE files outside April-October deleted')
# get a list with all available tiles
files = getFilelist(f'{base_path}{year}/tiles', '.tif', deep=True) 
unique_tiles = get_forceTSI_output_Tiles(files)
print(f'There are {len(unique_tiles)} tiles available for processing for the year {year}')

In [ ]:
# define the tiles to process and check if they contain composites for the same dates so that they can be stacked in the mosaiked
# (there is no sanity check for mosaiking)

tiles_to_process = unique_tiles#createFORCEtileLIST([67], [41])
date_list = []
for tile in tiles_to_process:
    date_list.append((get_forceTSI_output_DOYS([file for file in files if tile in file])))
for i in range(0,len(date_list)-1):
    if date_list[i] == date_list[i + 1]:
        continue
    else:
        print('the doys of composites across tiles is not equal - Better check!!!!!')
date_list = date_list[0]

In [ ]:
# get the LST file, aspect, ratio and incedence for the tile/time and stackem
# loop over the adequate bands from monthly LST stack (each band corresponds to the respective day of that month), asepct, ratio & incidence files

comp_stat = 'max'
LST_path = '/data/Aldhani/eoagritwin/et/Sentinel3/tiffs/LST/daily_observations_all/'

date = date_list[0] # will be replaced through loop

year = int(date[0:4])

# what bands (days) in LST from which month are associated with the current S2 scene
if is_leap_year(year):
    dayCountList = dayCount_leap
    end_day = 367
else:
    dayCountList = dayCount_noleap
    end_day = 366

day_sequence = [f'{year}{month:02d}{day:02d}' 
        for month, days_in_month in enumerate(dayCountList, start=1)
        for day in range(1, days_in_month + 1)]
    
dicti = {}

for i in range(1, end_day):
    month_idx = np.where(np.cumsum(dayCountList) >= i)[0][0]
    day_in_month =  i - (np.cumsum(dayCountList)[month_idx - 1] if month_idx > 0 else 0)

    dicti[i] = {
        'month': int_to_Month[f'{(month_idx + 1):02d}'],
        'band': day_in_month
    }

index_in_sequence = [i+1 for i, day in enumerate(day_sequence) if day == date][0]
print(index_in_sequence)

In [ ]:
# only working for growing season, does not work across years!!!!!!
for idx in range(index_in_sequence - 4, index_in_sequence + 5):
    if dicti[idx]['month'] in ['February', 'November']:
        continue
    else:
        print(dicti[idx]['month'])
        print(dicti[idx]['band'])
    #     ds = gdal.Open(f'/data/Aldhani/eoagritwin/et/Sentinel3/tiffs/LST/daily_observations_all/{year}/Daily_LST_means_year_{dicti[idx]['month']}.tif', 0)
    #     LST_arr = ds.GetRasterBand(dicti[idx]['band']) # store as single Tiff in temp

In [ ]:
# get dates from FORCE 10 day interpolations and make stacked tifs
for tile in tiles:
  for unique_day in unique_days:
    stack_tifs(sorted([file for file in files if unique_day in file and tile in file]), f'/data/{origin}force/output/Guzinski/1TileTest/{tile}_{unique_day}.tif')

In [ ]:
files_2Tiles = sorted([file for file in files for tile in tiles if tile in file and unique_days[12] in file])
files_2Tiles = [files_2Tiles[i] for i in [0,10,1,11,2,12,3,13,4,14,5,15,6,16,7,17,8,18]]


In [ ]:
files_2Tiles


In [ ]:
 # tiles = list(set([file.split('output/')[-1].split('/')[1].split('/')[0] for file in list_of_forcefiles]))
    

        # make paths in vrts relative
        vrts = getFilelist(outDir, '.vrt')
        for vrt in vrts:
            convertVRTpathsTOrelative(vrt)
        nums = [int(vrt.split('_')[-1].split('.')[0]) for vrt in vrts]
        vrts_sorted = sortListwithOtherlist(nums, vrts)[-1]
        print('paths in vrts made relative')
        
        vrt = gdal.BuildVRT(f'{outDir}{force_folder_name}_Cube.vrt', vrts_sorted, separate = True)
        vrt = None
        # convertVRTpathsTOrelative(f'{outDir}{force_folder_name}_Cube.vrt')
        print('overlord vrt created')
        if pyramids:
            # build pyramids
            vrtPyramids(f'{outDir}{force_folder_name}_Cube.vrt')
            print('VRT created with pyramids')
    else:
        print('Vrt might already exist - please check!!')

In [ ]:
highResFilename1 = '/data/Aldhani/eoagritwin/force/output/Guzinski/1TileTest/X0067_Y0042_20190630.tif'
highResFilename2 = '/data/Aldhani/eoagritwin/force/output/Guzinski/1TileTest/X0068_Y0042_20190630.tif'
lowResFilename = '/data/Aldhani/eoagritwin/et/Sentinel3/tiffs/LST/daily_observations_all/2019/Daily_LST_means_2019_July.tif'

outputFilename = '/data/Aldhani/eoagritwin/et/Sentinel3/tiffs/LST/test6_1.tif'

In [ ]:
useDecisionTree = True

commonOpts = {"highResFiles":               [highResFilename1, highResFilename2],#[highResFilename],
                "lowResFiles":              [lowResFilename, lowResFilename],
                "lowResQualityFiles":         [],# [lowResMaskFilename],
                "lowResGoodQualityFlags":     [],#[255],
                "cvHomogeneityThreshold":     0,
                "movingWindowSize":           15,
                "disaggregatingTemperature":  True}
dtOpts =     {"perLeafLinearRegression":    True,
                "linearRegressionExtrapolationRatio": 0.25}
sknnOpts =   {'hidden_layer_sizes':         (10,),
                'activation':                 'tanh'}
nnOpts =     {"regressionType":             REG_sklearn_ann,
                "regressorOpt":               sknnOpts}

start_time = time.time()

if useDecisionTree:
    opts = commonOpts.copy()
    opts.update(dtOpts)
    disaggregator = DecisionTreeSharpener(**opts)
else:
    opts = commonOpts.copy()
    opts.update(nnOpts)
    disaggregator = NeuralNetworkSharpener(**opts)

In [ ]:
print("Training regressor...")
disaggregator.trainSharpener()
print("Sharpening...")
downscaledFile = disaggregator.applySharpener(highResFilename1, lowResFilename)
print("Residual analysis...")
residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                               # lowResMaskFilename,
                                                                doCorrection=True)
print("Saving output...")
highResFile = gdal.Open(highResFilename1)
if correctedImage is not None:
    outImage = correctedImage
else:
    outImage = downscaledFile
# outData = utils.binomialSmoother(outData)
outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                        outImage.GetGeoTransform(),
                        outImage.GetProjection(),
                        outputFilename)
residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                            residualImage.GetGeoTransform(),
                            residualImage.GetProjection(),
                            os.path.splitext(outputFilename)[0] + "_residual" +
                            os.path.splitext(outputFilename)[1])

outFile = None
residualFile = None
downsaceldFile = None
highResFile = None

print(time.time() - start_time, "seconds")